# Notebook for pysc2 environment exploration
The python api for starcraft 2 is not as simple as many of the openai gym learning environments. 

Luckily there's a mock env for testing the code in a notebook like this.

### Imports

In [1]:
from pysc2.env.mock_sc2_env import SC2TestEnv
from pysc2.env import sc2_env
from pysc2.agents import base_agent
from pysc2.lib import actions

import numpy as np
import json

%load_ext autoreload
%autoreload 2

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


### Initialize pysc2 mocv env

In [2]:
players = [
    sc2_env.Agent(race=sc2_env.Race.terran),
    sc2_env.Agent(race=sc2_env.Race.protoss),
]

kwargs = {
    'agent_interface_format': [
        sc2_env.AgentInterfaceFormat(
            feature_dimensions=sc2_env.Dimensions(
                screen=(8, 8),
                minimap=(4, 4)
            ),
            rgb_dimensions=sc2_env.Dimensions(
                screen=(63, 63),
                minimap=(15, 15)
            ),
            action_space=sc2_env.ActionSpace.FEATURES
        ),
    ]
}


env = SC2TestEnv(**kwargs)

### Exploration of the timestep returned from the env

In [3]:
time_step = env.reset()

# for now we will only get one observation at a time
step_type, reward, discount, observation = time_step[0]

print(step_type)
print(reward)
print(discount)
print(type(observation.feature_screen))

indexes = [
    'height_map', 
    'visibility_map', 
    'creep', 
    'power', 
    'player_id', 
    '...', 
    'unit_shields', 
    'unit_shields_ratio', 
    'unit_density', 
    'unit_density_aa', 
    'effects'
]


print(observation.available_actions)

action_spec = env.action_spec()


#action_spec.functions

#actions.FunctionCall(2, )


#print(type(obs[0].observation))
#print(type(obs[0]))
#print(json.dumps(obs[0].observation))

StepType.FIRST
0.0
0.0
<class 'pysc2.lib.named_array.NamedNumpyArray'>
[  0   1   2   3   4 451   6   7 452]


In [4]:
class RandomAgent(base_agent.BaseAgent):
  """A random agent for starcraft."""

  def step(self, obs):
    super(RandomAgent, self).step(obs)
    function_id = numpy.random.choice(obs.observation.available_actions)
    args = [[numpy.random.randint(0, size) for size in arg.sizes]
            for arg in self.action_spec.functions[function_id].args]
    return actions.FunctionCall(function_id, args)


### Define the agent

In [5]:
class BaseAgent(object):
  """A base agent to write custom scripted agents.

  It can also act as a passive agent that does nothing but no-ops.
  """

  def __init__(self):
    self.reward = 0
    self.episodes = 0
    self.steps = 0
    self.obs_spec = None
    self.action_spec = None

  def setup(self, obs_spec, action_spec):
    self.obs_spec = obs_spec
    self.action_spec = action_spec

  def reset(self):
    self.episodes += 1

  def step(self, obs):
    self.steps += 1
    self.reward += obs.reward
    return actions.FunctionCall(actions.FUNCTIONS.no_op.id, [])

In [6]:
class MyAgent(BaseAgent):
    
    def step(self, obs):
        super(MyAgent, self).step(obs)
        